In [1]:
from sklearn.preprocessing import MinMaxScaler
import itertools
import pandas as pd
import torch

# get our predefined functions (for data processing, etc.)
import QCircNet.utils as ut

# get our predefined quantum circuits and neural networks
import QCircNet.circuits.CustomCircuits as cn
import QCircNet.BinaryQuantumNN as qnn


EXPERIMENT_ID = 0
HYBRID = True

X_PATH = "data/X_apd_class6_7.npy"
Y_PATH = "data/y_apd_class6_7.npy"
BIN_ENCODING = {6: 0, 7: 1}


N_QUBITS = 4
FEAT_PER_QUBIT = 4

SCALER = MinMaxScaler()

SUBSET = None

EPOCHS = 10
WEIGHT_PATH = f"results/weights/bin/"
MODEL_PATH = f"results/models/bin/"
RESULTS_PATH = f"results/bin_model_results_expID_{EXPERIMENT_ID}.csv"
RESULTS_DF = pd.DataFrame()

VALIDATION = True

# for printing colored text output during training
RED_TEXT = "\033[91m"
GREEN_TEXT = "\033[92m"
RESET_COLOR = "\033[0m" 


In [2]:
# load the data
X_train, X_val, X_test, y_train, y_val, y_test, _ = ut.load_and_prepare_data(X_PATH, Y_PATH, scaler=SCALER, bin_encoding=BIN_ENCODING, subset=SUBSET)

# check shapes
print("train:\t", X_train.shape, y_train.shape)
print("val:\t", X_val.shape, y_val.shape)
print("test:\t", X_test.shape, y_test.shape)

train:	 torch.Size([9746, 16]) torch.Size([9746])
val:	 torch.Size([1219, 16]) torch.Size([1219])
test:	 torch.Size([1218, 16]) torch.Size([1218])


In [3]:
cn.get_custom_circuits()

[QCircNet.circuits.CustomCircuits.Anusha2024Circuit,
 QCircNet.circuits.CustomCircuits.DoubleEntanglementVLCircuit,
 QCircNet.circuits.CustomCircuits.Ranga2024Circuit,
 QCircNet.circuits.CustomCircuits.Senokosov2024Circuit]

In [4]:
circuits = [
    cn.Ranga2024Circuit,
    cn.Senokosov2024Circuit,
    cn.Anusha2024Circuit,
    cn.DoubleEntanglementVLCircuit,
]

runs = list(range(1, 11)) # runs are also used as seeds (reproducibility)
learning_rates = [0.01, 0.001]

In [ ]:
### RUN EXPERIMENT
id = 0
for seed, circuit, lr in itertools.product(runs, circuits, learning_rates):
    print(f"""{RED_TEXT}RUN {id+1}:{RESET_COLOR}\n {circuit.__name__} \n lr: {lr}""")

    # init the model
    model = qnn.BinQuantumNeuralNetwork(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=seed, hybrid=HYBRID)

    path_identifier = f"{circuit.__name__}_expID_{EXPERIMENT_ID}_dfID_{id}"

    # train the model (predifined optimizer, etc.)
    trained_model, loss_history, avg_epoch_time = ut.train_model(
        model, X_train, y_train,
        epochs=EPOCHS, batch_size=32, lr=lr, binary=True
    )
    
    # save trained model
    model_path = MODEL_PATH+path_identifier+".pth"
    torch.save(trained_model, model_path)

    # evaluate
    cross_entropy, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)

    results_dict = {
        "id": id,
        "f1_score": f1, "precision": precision, "recall": recall, "accuracy": accuracy, "cross_entropy": cross_entropy,
        "seed": seed,
        "circuit": circuit.__name__,
        "learning_rate": lr,
        "loss_history": loss_history,
        "avg_epoch_time_seconds": avg_epoch_time,
        "model_path": model_path,
        "epochs": EPOCHS,
        "n_qubits": N_QUBITS,
        "features_per_qubit": FEAT_PER_QUBIT,
        "scaler" : str(SCALER).replace("()", "")
    }
    print(f"\n{GREEN_TEXT}RESULTS:\n F1-Score:{f1}, Precision:{precision}, Recall:{recall}, Accuracy:{accuracy}, Loss:{cross_entropy} {RESET_COLOR} \n\n")
    
    # append the results_dict to the RESULTS_DF
    RESULTS_DF = pd.concat([RESULTS_DF, pd.DataFrame([results_dict])], ignore_index=True)
    RESULTS_DF.to_csv(path_or_buf=RESULTS_PATH) # save for each run (in case of a crash)

    id += 1

RUN 1:
 Ranga2024Circuit 
 lr: 0.01


Training progress: 100%|██████████| 304/304 [02:40<00:00,  1.89batch/s]


Epoch 1/10, Train-Loss: 0.1195


Training progress: 100%|██████████| 304/304 [02:39<00:00,  1.91batch/s]


Epoch 2/10, Train-Loss: 0.0289


...

Training progress: 100%|██████████| 304/304 [02:48<00:00,  1.81batch/s]


Epoch 3/10, Train-Loss: 0.0999


Training progress: 100%|██████████| 304/304 [02:52<00:00,  1.76batch/s]


Epoch 4/10, Train-Loss: 0.0670


Training progress:  68%|██████▊   | 207/304 [17:07<00:55,  1.74batch/s]   

In [ ]:
RESULTS_DF

,id,f1_score,precision,recall,accuracy,cross_entropy,seed,circuit,learning_rate,loss_history,val_losses,val_f1s,avg_epoch_time_seconds,network_str,model_path,weight_path,epochs,n_qubits,features_per_qubit,scaler
0,0,0.993579,0.998387,0.988818,0.993432,0.023789,1,Ranga2024Circuit,0.010,"[0.1194998185291249, 0.02889960845028614]","[0.03633466362953186, 0.02427237667143345]","[0.9920382165605095, 0.9928172386272945]",209.200972,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Ranga2024Circuit_expID_0_df...,results/weights/bin/,2,4,4,MinMaxScaler
1,1,0.990354,0.996764,0.984026,0.990148,0.146723,1,Ranga2024Circuit,0.001,"[0.465129475452398, 0.1997821974862171]","[0.28358474373817444, 0.14489339292049408]","[0.9904306220095693, 0.9935897435897436]",190.780761,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Ranga2024Circuit_expID_0_df...,results/weights/bin/,2,4,4,MinMaxScaler
2,2,0.993569,1.000000,0.987220,0.993432,0.028628,1,Senokosov2024Circuit,0.010,"[0.15520546564795568, 0.02918960083901]","[0.042100317776203156, 0.02745809778571129]","[0.9912490055688147, 0.9952]",159.079280,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Senokosov2024Circuit_expID_...,results/weights/bin/,2,4,4,MinMaxScaler
3,3,0.991987,0.995177,0.988818,0.991790,0.205413,1,Senokosov2024Circuit,0.001,"[0.5929172167456463, 0.2740683712457356]","[0.38416343927383423, 0.20542927086353302]","[0.9849802371541502, 0.9880668257756563]",158.450551,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Senokosov2024Circuit_expID_...,results/weights/bin/,2,4,4,MinMaxScaler
4,4,0.993579,0.998387,0.988818,0.993432,0.027842,1,Anusha2024Circuit,0.010,"[0.13495928062260837, 0.032840807977328565]","[0.04125167429447174, 0.02655337192118168]","[0.9936204146730463, 0.9944044764188649]",62.548607,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Anusha2024Circuit_expID_0_d...,results/weights/bin/,2,4,4,MinMaxScaler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,0.990369,0.995161,0.985623,0.990148,0.094961,10,Senokosov2024Circuit,0.001,"[0.4246948525113495, 0.1296829528812515]","[0.19592657685279846, 0.0956486240029335]","[0.9856459330143541, 0.9920382165605095]",157.125579,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Senokosov2024Circuit_expID_...,results/weights/bin/,2,4,4,MinMaxScaler
76,76,0.992794,0.995185,0.990415,0.992611,0.033130,10,Anusha2024Circuit,0.010,"[0.12039245833569255, 0.03125268833766887]","[0.03968925401568413, 0.03411133959889412]","[0.994413407821229, 0.9920508744038156]",62.042147,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Anusha2024Circuit_expID_0_d...,results/weights/bin/,2,4,4,MinMaxScaler
77,77,0.990354,0.996764,0.984026,0.990148,0.076691,10,Anusha2024Circuit,0.001,"[0.38039971476322726, 0.09725551064567346]","[0.13609278202056885, 0.07773631066083908]","[0.9928172386272945, 0.9920255183413078]",62.107158,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/Anusha2024Circuit_expID_0_d...,results/weights/bin/,2,4,4,MinMaxScaler
78,78,0.984177,0.974922,0.993610,0.983580,0.033319,10,DoubleEntanglementVLCircuit,0.010,"[0.13361566995062227, 0.024704898431082256]","[0.03278356418013573, 0.0341801792383194]","[0.994413407821229, 0.9897070467141726]",90.930680,BinQuantumNeuralNetwork(\n (quantum_circuit_n...,results/models/bin/DoubleEntanglementVLCircuit...,results/weights/bin/,2,4,4,MinMaxScaler
